In [1]:
import numpy as np

# ------------------------------
# Example Inputs: Closing prices
# ------------------------------
# These should be aligned and cleaned already
prices_A = np.array([100, 101, 102, 103, 105, 104, 106])
prices_B = np.array([200, 198, 199, 202, 201, 205, 208])

# -------------------------------------
# Step 1: Convert to log returns
# -------------------------------------
# Formula: log(P_t) - log(P_{t-1})
log_returns_A = np.diff(np.log(prices_A))
log_returns_B = np.diff(np.log(prices_B))

# Both return arrays are now of length len(prices) - 1
assert len(log_returns_A) == len(log_returns_B), "Return arrays must be equal length"

# -------------------------------------
# Step 2: Compute Pearson Correlation
# -------------------------------------
# Manual computation
mean_A = np.mean(log_returns_A)
mean_B = np.mean(log_returns_B)

covariance = np.mean((log_returns_A - mean_A) * (log_returns_B - mean_B))
std_A = np.std(log_returns_A)
std_B = np.std(log_returns_B)

pearson_correlation = covariance / (std_A * std_B)

# Or, using NumPy's built-in method:
pearson_np = np.corrcoef(log_returns_A, log_returns_B)[0, 1]

# -------------------------------------
# Output
# -------------------------------------
print("Manual Pearson Correlation:", round(pearson_correlation, 4))
print("NumPy Pearson Correlation :", round(pearson_np, 4))

Manual Pearson Correlation: -0.4657
NumPy Pearson Correlation : -0.4657
